In [1]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 40 kB 1.7 MB/s eta 0:00:011
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.4-py3-none-any.whl size=30114 sha256=8add6b6b4598d4c300f4814767ee1440b896ccacbd5ae05a428af27aa710a1a0
  Stored in directory: /home/nobu_yamaguchi/.cache/pip/wheels/6c/c9/9c/363182ea34a736dae336eeaf0dd4a7eec3c6a5afe32373e1fe
  Created wheel for py-params: filename=py_params-0.9.7-py3-none-any.whl size=7302 sha256=8ac0dcbc968ce8599110cba6d6bb23899f4465fb7eea4584186b35556c449a0a
  Stored in directory: /home/nobu_yamaguchi/.cache/pip/wheels/47/3d/2d/bbffcfd6b9f4b8b5cbf07e7520ac2676192fe9431240c13ee8
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19473 sha256=db82dd4d94cafc56d4ee6e075c4dc30fc1c29e662d0b70567d39fb5248e18166
  Stored in directory: /home/nobu_yamaguchi/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
Successfully built bert-for-tf2 py-params params-flow
You should consider upgrading via 

In [4]:
!pip install tensorflow_hub

     |████████████████████████████████| 101 kB 2.7 MB/s ta 0:00:011
You should consider upgrading via the '/home/nobu_yamaguchi/anaconda3/bin/python -m pip install --upgrade pip' command.


In [1]:
import tensorflow as tf

import tensorflow_hub as hub
import numpy as up
import pandas as pd
from tensorflow.keras import layers
import bert

In [2]:
import numpy as np
import pandas as pd
import pickle as cPickle
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os
import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

Using TensorFlow backend.


In [3]:
df_merge_quality = pd.read_csv('US_patent_abstract_5000_2015_with_title_1.csv')
df_merge_quality.shape

(5000, 30)

In [4]:
df = df_merge_quality[['claims_text', 'quality_rank']]

In [5]:
df.quality_rank.value_counts()

0    2896
1    2104
Name: quality_rank, dtype: int64

In [6]:
2896/5000

0.5792

In [7]:
df.shape

(5000, 2)

In [8]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [9]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [10]:
claims = []
sentences = list(df['claims_text'])
for sen in sentences:
    claims.append(preprocess_text(str(sen)))

In [11]:
print(df.columns.values)

['claims_text' 'quality_rank']


In [12]:
df.quality_rank.unique()

array([0, 1])

In [13]:
y = df.quality_rank

In [14]:
y[10]

1

In [15]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [16]:
def tokenize_claims(text_claims):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_claims))

In [17]:
tokenized_claims = [tokenize_claims(claim) for claim in claims]

Prerparing Data For Training

In [18]:
claims_with_len = [[claim, y[i], len(claim)]
                 for i, claim in enumerate(tokenized_claims)]

In [22]:
#np.random.shuffle(claims_with_len)

In [28]:
#claims_with_len.sort(key=lambda x: x[2])

In [19]:
sorted_claims_labels = [(claim_lab[0], claim_lab[1]) for claim_lab in claims_with_len]

In [20]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_claims_labels, output_types=(tf.int32, tf.int32))

In [21]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [22]:
next(iter(batched_dataset))

(<tf.Tensor: shape=(32, 4552), dtype=int32, numpy=
 array([[ 5080,  9605,  3638, ...,     0,     0,     0],
        [ 1996, 11028,  3555, ...,     0,     0,     0],
        [ 2054,  2003,  3555, ...,     0,     0,     0],
        ...,
        [ 2019,  1999,  4946, ...,     0,     0,     0],
        [ 1996, 11028,  3555, ...,     0,     0,     0],
        [ 2054,  2003,  3555, ...,     0,     0,     0]], dtype=int32)>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1,
        1, 0, 0, 0, 1, 1, 0, 0, 1, 1], dtype=int32)>)

In [23]:
TOTAL_BATCHES = np.math.ceil(len(sorted_claims_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 20
#batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [24]:
a = 0
for element in test_data:
    a += len(element[1])
a

224

In [25]:
i = 0
for element in test_data:
    i += sum(element[1])
i

<tf.Tensor: shape=(), dtype=int32, numpy=100>

In [40]:
1-48/224

0.7857142857142857

In [33]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=200,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.4,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [34]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.4

NB_EPOCHS = 5

In [35]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [36]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["acc"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

In [37]:
text_model.fit(train_data, epochs=NB_EPOCHS, validation_data=test_data)

Epoch 1/5
150/150 [==============================] - 256s 2s/step - loss: 0.6783 - acc: 0.5808 - val_loss: 0.6711 - val_acc: 0.6250
Epoch 2/5
150/150 [==============================] - 255s 2s/step - loss: 0.6048 - acc: 0.6671 - val_loss: 0.6385 - val_acc: 0.6027
Epoch 3/5
150/150 [==============================] - 254s 2s/step - loss: 0.3562 - acc: 0.8576 - val_loss: 0.8478 - val_acc: 0.6027
Epoch 4/5
150/150 [==============================] - 254s 2s/step - loss: 0.0880 - acc: 0.9780 - val_loss: 1.0022 - val_acc: 0.6027
Epoch 5/5
150/150 [==============================] - 254s 2s/step - loss: 0.0304 - acc: 0.9946 - val_loss: 1.2334 - val_acc: 0.5893


In [46]:
#0.3
results = text_model.evaluate(test_data)
print(results)

      7/Unknown - 0s 68ms/step - loss: 1.7654 - acc: 0.4062[1.7654307399477278, 0.40625]


In [34]:
#0.3
results = text_model.evaluate(test_data)
print(results)

      7/Unknown - 0s 40ms/step - loss: 1.2583 - acc: 0.5982[1.2583002362932478, 0.59821427]


In [124]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/5
150/150 [==============================] - 14s 95ms/step - loss: 0.6814 - acc: 0.5756
Epoch 2/5
150/150 [==============================] - 13s 89ms/step - loss: 0.6092 - acc: 0.6654
Epoch 3/5
150/150 [==============================] - 13s 86ms/step - loss: 0.2435 - acc: 0.9206
Epoch 4/5
150/150 [==============================] - 13s 87ms/step - loss: 0.0681 - acc: 0.9765
Epoch 5/5
150/150 [==============================] - 13s 87ms/step - loss: 0.0296 - acc: 0.9923


In [ ]:
pred = model.predict(x_val)
pred_output = []

for cnt in range(2000):
    if pred[cnt][0] > pred[cnt][1]: 
        pred_output.append(0)
    else:
        pred_output.append(1)

[i for i in pred_output if pred_output[i] != 0]

In [105]:
#0.2
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/5
150/150 [==============================] - 14s 90ms/step - loss: 0.6806 - acc: 0.5775
Epoch 2/5
150/150 [==============================] - 13s 86ms/step - loss: 0.6067 - acc: 0.6650
Epoch 3/5
150/150 [==============================] - 13s 86ms/step - loss: 0.2338 - acc: 0.9223
Epoch 4/5
150/150 [==============================] - 13s 86ms/step - loss: 0.0501 - acc: 0.9858
Epoch 5/5
150/150 [==============================] - 13s 86ms/step - loss: 0.0124 - acc: 0.9987


In [106]:
# Save the trained model
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [107]:
results = text_model.evaluate(test_data)
print(results)

      7/Unknown - 0s 28ms/step - loss: 1.2172 - acc: 0.6116[1.2172341517039709, 0.61160713]


In [126]:
test_data[0]

TypeError: 'TakeDataset' object is not subscriptable

In [ ]:
# predict probabilities for test set
yhat_probs = text_model.predict(test_data, verbose=0)
# predict crisp classes for test set
yhat_classes = text_model.predict_classes(test_data, verbose=0)
# reduce to 1d array
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]
 
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(testy, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(testy, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(testy, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(testy, yhat_classes)
print('F1 score: %f' % f1)
 
# kappa
kappa = cohen_kappa_score(testy, yhat_classes)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = roc_auc_score(testy, yhat_probs)
print('ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(testy, yhat_classes)
print(matrix)

In [58]:
type(test_data)

tensorflow.python.data.ops.dataset_ops.TakeDataset

In [56]:
#0.1
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/5
150/150 [==============================] - 14s 91ms/step - loss: 0.6804 - acc: 0.5771
Epoch 2/5
150/150 [==============================] - 13s 87ms/step - loss: 0.6038 - acc: 0.6600
Epoch 3/5
150/150 [==============================] - 12s 82ms/step - loss: 0.2177 - acc: 0.9324
Epoch 4/5
150/150 [==============================] - 13s 85ms/step - loss: 0.0468 - acc: 0.9858
Epoch 5/5
150/150 [==============================] - 13s 86ms/step - loss: 0.0134 - acc: 0.9975


In [57]:
results = text_model.evaluate(test_data)
print(results)

      7/Unknown - 0s 24ms/step - loss: 1.2959 - acc: 0.6071[1.295912163598197, 0.60714287]
